# Setup

In [1]:
# !pip install --upgrade gradio
# !pip install fpdf

from fpdf import FPDF
import gradio as gr
import json
import os

/Users/Shared/Main/CMU/Assignments/DSHS/project/90-835-Team-Project/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# pip install openai==1.12.0

In [3]:
import openai
from openai import OpenAI

print(openai.__version__) # version - 1.12.0


1.75.0


### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

# Load data and credentials

In [4]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.
import json
credentials_path = 'credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


In [ ]:
import json

# Update the path if your file is in a different location
file_path = 'data/data.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)

# To verify, print a part of the data
patient_data.pop('patient_id', None)
if 'name' in patient_data['patient_demographics']:
    patient_data['patient_demographics'].pop('name', None)

print(patient_data['patient_demographics'])

{'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}


# Explore data

In [6]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [7]:
print(f'Keys in data: {(patient_data.keys())}')

Keys in data: dict_keys(['patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])


In [8]:
# Look at encounters data, there are multiple
patient_data['encounters']

[{'date': '2024-02-10',
  'type': 'Admission',
  'reason': 'Cough, shortness of breath, hemoptysis, fever'},
 {'date': '2024-02-14', 'type': 'Discharge'}]

In [9]:
# Look at flowsheets data, there are multiple
patient_data['flowsheets']

[{'date': '2024-02-10',
  'time': '08:00',
  'temperature': '38.5°C',
  'heart_rate': '90 bpm',
  'blood_pressure': '130/85 mmHg',
  'respiratory_rate': '20 breaths/min',
  'oxygen_saturation': '92%'},
 {'date': '2024-02-11',
  'time': '08:00',
  'temperature': '37.8°C',
  'heart_rate': '85 bpm',
  'blood_pressure': '125/80 mmHg',
  'respiratory_rate': '18 breaths/min',
  'oxygen_saturation': '94%'},
 {'date': '2024-02-12',
  'time': '08:00',
  'temperature': '37°C',
  'heart_rate': '80 bpm',
  'blood_pressure': '120/75 mmHg',
  'respiratory_rate': '16 breaths/min',
  'oxygen_saturation': '96%'},
 {'date': '2024-02-13',
  'time': '08:00',
  'temperature': '36.5°C',
  'heart_rate': '78 bpm',
  'blood_pressure': '118/74 mmHg',
  'respiratory_rate': '14 breaths/min',
  'oxygen_saturation': '98%'}]

In [10]:
patient_data['notes']

[{'date': '2024-02-10',
  'author': 'Dr. Smith',
  'note_type': 'Admission Note',
  'content': 'Patient admitted with symptoms indicative of pneumonia. Initiated empirical antibiotic therapy.'},
 {'date': '2024-02-11',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient shows signs of improvement. CRP levels declining. Continue with current antibiotic regimen. Monitor for any adverse reactions.'},
 {'date': '2024-02-12',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': "Patient's temperature normalized. Oxygen saturation levels stable. Plan to switch to oral antibiotics tomorrow."},
 {'date': '2024-02-13',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient continues to improve. CRP and WBC trending down. Ready for discharge planning.'},
 {'date': '2024-02-14',
  'author': 'Dr. Smith',
  'note_type': 'Discharge Summary',
  'content': "Patient's condition improved with treatment. Medically fit for discharge

# Sample zero-shot solution
This is a first attempt where we pass in everything and provide basic prompt engineering. This is a baseline solution. Your solutions should improve from this.

In [11]:
client = OpenAI(api_key=openai_api_key)

In [12]:
# def generate_patient_summary_v1(patient_data, additional_prompts=''):
#     """
#     Generates a patient summary using OpenAI's GPT-3.5.

#     Parameters:
#     - patient_data (str): The patient data to generate the summary from.
#     - additional_prompts (str): Try prompt engineering here.

#     Returns:
#     - str: The generated patient summary.
#     """

#     # Initialize the OpenAI client with your API key
#     client = OpenAI(api_key=openai_api_key)

#     # Create the chat completion request with the patient data
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": f"""
#                               You are an experienced and detail-oriented physician writing a discharge summary for another clinician.
# Use the structured patient data provided to create a well-written, narrative-style discharge letter that clearly describes the patient’s hospitalization.
# 	The summary should be written in paragraph form — avoid lists or bullet points.

# 	✍️ Discharge Summary Guidelines:
# 		•	Begin with the presentation: why the patient came in, symptoms on arrival, relevant exam findings.
# 	•	Describe key investigations: include only significant test results or pending follow-ups.
# 	•	State the final diagnosis (include specificity, e.g., “bacterial pneumonia,” not just “pneumonia”).
# 	•	Describe the hospital course in chronological order: treatment, response, medication changes, and progress.
# 	•	Mention any complications that occurred and how they were managed or resolved.
# 	•	Document any procedures performed, including the date and relevant outcomes.
# 	•	Provide a clear discharge plan: current medications, outstanding issues, and follow-up instructions.
# 	•	Conclude with a brief comment on the patient’s condition at discharge (functional status, vitals, mobility).

# 	Ensure the summary:
# 		•	Uses professional clinical language
# 	•	Is easy to read and logically structured
# 	•	Includes specific and relevant clinical data
# 	•	Clearly outlines follow-up needs for continuity of care
    
# 	Example Output Style:

# 	“Mr. Smith was admitted on March 3rd with worsening breathlessness and productive cough. On arrival, he was febrile and hypoxic with left lower lobe crackles. Chest X-ray confirmed left lower lobe consolidation. He was diagnosed with community-acquired pneumonia and started on IV antibiotics. Over the next four days, he showed good clinical improvement with normalization of CRP and WBC levels. Antibiotics were stepped down to oral form on Day 4. He was discharged home on Day 5 with instructions to complete a 5-day course of amoxicillin and follow up with his GP in one week…”
#                               {additional_prompts}
#                               Data: {patient_data}
#                 """,
#             }
#         ],
#         model="gpt-3.5-turbo",
#     )

#     # Return the content of the generated message
#     return chat_completion.choices[0].message.content

In [13]:
def generate_patient_summary(patient_data, additional_prompts=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Try prompt engineering here.

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                              You are a clinical language model tasked with generating a hospital discharge summary from patient data.

Your goal is to write a clear, professional discharge summary in a narrative paragraph form, as if written by a physician for a real-world care transition.

IMPORTANT INSTRUCTIONS:

•⁠  ⁠If the patient’s "discharge_ready" field is false or not present, DO NOT produce any output (return an empty result).
•⁠  ⁠If "discharge_ready" is true, proceed to generate the discharge summary.
•⁠  ⁠Under no circumstance should the summary include any personally identifiable information (PII). Do NOT include:
  - Patient’s full name
  - Date of birth
  - Medical record number
  - Addresses
  - Contact details
Refer to the individual only as "the patient" or "this patient."

DISCHARGE SUMMARY REQUIREMENTS:

Write the summary using natural, flowing paragraphs — not lists or bullet points. Include the following:

•	Begin with the presentation: why the patient came in, symptoms on arrival, relevant exam findings.
	•	Describe key investigations: include only significant test results or pending follow-ups.
	•	State the final diagnosis (include specificity, e.g., “bacterial pneumonia,” not just “pneumonia”).
	•	Describe the hospital course in chronological order: treatment, response, medication changes, and progress.
	•	Mention any complications that occurred and how they were managed or resolved.
	•	Document any procedures performed, including the date and relevant outcomes.
	•	Provide a clear discharge plan: current medications, outstanding issues, and follow-up instructions.
	•	Conclude with a brief comment on the patient’s condition at discharge (functional status, vitals, mobility).
Ensure the summary:
		•	Uses professional clinical language
	•	Is easy to read and logically structured
	•	Includes specific and relevant clinical data
	•	Clearly outlines follow-up needs for continuity of care
    
STYLE & TONE:

•⁠  ⁠Use a formal, clinical tone and full sentences.
•⁠  ⁠Keep the summary focused, clear, and easy to interpret for other providers.
•⁠  ⁠Organize information in a logical narrative: start with diagnosis and admission, followed by hospital course, and conclude with discharge planning.
•⁠  ⁠Again, do NOT include any patient identifiers — respect privacy at all times.

Now, based on the patient_json input, if the patient is cleared for discharge, generate the discharge summary in full paragraphs. If the patient is not ready for discharge, output nothing.

                              {additional_prompts}
                              Data: {patient_data}
                """,
            }
        ],
        model="gpt-3.5-turbo",
    )

    # Return the content of the generated message
    return chat_completion.choices[0].message.content

In [14]:
print(generate_patient_summary(patient_data))

Discharge Summary:

The patient, a 70-year-old male, was admitted on 10th February 2024 presenting with symptoms of cough, shortness of breath, hemoptysis, and fever. Initial examination revealed a temperature of 38.5°C, heart rate of 90 bpm, blood pressure of 130/85 mmHg, respiratory rate of 20 breaths/min, and oxygen saturation of 92%. Chest X-ray showed consolidation in the left lower lobe, consistent with the diagnosis of lobar pneumonia, unspecified organism.

Key investigations indicated an elevated CRP of 60 mg/L and a WBC count of 12 x10^9/L on admission, supporting the diagnosis. The patient was promptly started on Amoxicillin IV along with supportive care. Over the hospital course, the patient showed gradual improvement with decreasing inflammatory markers and stable vital signs.

By the 13th of February, the patient's condition had significantly improved with CRP levels down to 20 mg/L, WBC reduced to 7 x10^9/L, and vital signs within normal limits. It was deemed appropriate

In [15]:
# Same code, second run; notice difference as a result of stochastic process
print(generate_patient_summary(patient_data))

Upon admission on February 10, 2024, the patient, a 70-year-old male, presented with cough, shortness of breath, hemoptysis, and fever. Diagnostic workup revealed lobar pneumonia with consolidation in the left lower lobe on chest X-ray. Laboratory findings showed elevated CRP levels at 60 mg/L and a white blood cell count of 12 x10^9/L. The patient was diagnosed with lobar pneumonia, unspecified organism, and initiated on IV Amoxicillin.

Throughout the hospital course, the patient responded well to treatment with decreasing inflammatory markers and stable vital signs. On February 13, the patient transitioned to oral antibiotics and showed significant clinical improvement with decreasing CRP levels and WBC count. With the patient's condition stabilized, it was deemed appropriate to prepare for discharge.

No significant complications were encountered during the hospitalization, and the patient was managed effectively with the prescribed antibiotic regimen. At the time of discharge on F

In [16]:
# Some counterproductive prompt engineering
print(generate_patient_summary(patient_data, """
You are a clinical assistant tasked with creating a high-quality discharge summary for a patient who recently completed an inpatient hospital stay. Your goal is to produce a medically accurate, well-organized, and concise document that can be used directly by receiving providers for continued patient care.

Use the structured patient data in the input to write a discharge summary that adheres to professional standards outlined by clinical guides such as those on Geeky Medics and from hospital best practices.

Follow these steps in your reasoning before generating the final summary:

Step 1: Read the patient data and identify key elements:
  - What is the primary diagnosis (not just a symptom)?
  - What significant history impacts post-discharge care?
  - What medications are current and which were adjusted?
  - What major procedures or tests occurred?
  - What events defined the hospital course?
  - What tasks or checks remain post-discharge?
  - Are there any scheduled follow-up appointments?

Step 2: Organize the discharge summary using these standard sections:
  - Patient Details (Name, Age, MRN if available)
  - Reason for Admission
  - Relevant Past Medical History
  - Hospital Course
  - Significant Tests & Procedures
  - Medications at Discharge (and any changes)
  - Allergies
  - Outstanding Issues
  - Follow-Up and Appointments

Step 3: Maintain a formal, clinical tone. Be precise but concise. Only include information that supports continued care.

Output only the final formatted discharge summary, without revealing the reasoning steps.
"""))

Discharge Summary:

The patient, a 70-year-old male, was admitted on February 10, 2024, presenting with symptoms of cough, shortness of breath, hemoptysis, and fever, consistent with a diagnosis of lobar pneumonia, unspecified organism. Initial assessments revealed consolidation in the left lower lobe on a chest X-ray. The patient's hospital course was marked by empirical antibiotic treatment with Amoxicillin IV and supportive care, showing gradual improvement in clinical parameters.

Key laboratory findings displayed a decreasing trend in CRP levels from 60 mg/L on admission to 20 mg/L before discharge, along with a progressive decline in WBC count and stable hemoglobin and platelet levels. Vital signs improved over the hospital stay, with a gradual normalization of temperature and oxygen saturation reaching 98%.

In terms of medications, the patient was transitioned from intravenous to oral Amoxicillin, along with continued Atorvastatin for cholesterol management. No new allergies we

In [17]:
def load_patient_data(json_file):
    """Load patient data from uploaded JSON file and return formatted patient list for dropdown"""
    try:
        # Read the JSON content
        if json_file is None:
            return [], {}

        content = json_file.decode('utf-8')
        patients_data = json.loads(content)

        # Create formatted choices for dropdown: "ID: Name"
        patient_choices = []
        patient_data_dict = {}

        for patient in patients_data:
            patient_id = patient.get("patient_id", "Unknown")
            patient_name = patient.get("patient_demographics", {}).get("name", "Unknown")
            display_text = f"{patient_id}: {patient_name}"
            patient_choices.append(display_text)
            patient_data_dict[display_text] = patient

        return patient_choices, patient_data_dict
    except Exception as e:
        print(f"Error loading patient data: {e}")
        return [], {}

In [18]:
def generate_pdf(patient_selection, custom_message, patient_data_dict):
    """Generate a PDF discharge letter for the selected patient"""
    try:
        if not patient_selection or patient_selection not in patient_data_dict:
            return None

        patient_data = patient_data_dict[patient_selection]
        # letter_content = generate_discharge_letter(patient_data, custom_message)
        letter_content = generate_patient_summary(patient_data, custom_message)

        # Create a PDF instance
        pdf = FPDF()
        pdf.add_page()

        # Add title with larger font
        pdf.set_font("Times", "B", size=24)
        pdf.cell(200, 20, txt="Discharge Letter", ln=True, align='C')

        # Add the content to the PDF
        pdf.set_font("Times", size=12)
        pdf.multi_cell(0, 10, txt=letter_content)

        # Add footer
        pdf.set_y(0)
        pdf.set_font("Times", "I", size=10)
        pdf.cell(200, 10, txt="256 Baker Street, Pune, India | +1 (412)844-1891 | Micro Mini Hospitals Pvt. Ltd.", ln=True, align='C')

        # Save the PDF to a temporary file
        output_path = "discharge_letter.pdf"
        pdf.output(output_path)

        return output_path
    except Exception as e:
        print(f"Error generating PDF: {e}")
        return None

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Patient Discharge Letter Generator")

    # Store patient data in state
    patient_data_dict = gr.State({})

    with gr.Row():
        json_file = gr.File(
            label="Upload Patient JSON Data",
            file_types=[".json"],
            type="binary"
        )

    with gr.Row():
        patient_dropdown = gr.Dropdown(
            choices=[],
            label="Select a patient",
            value=None,
            interactive=True
        )

    with gr.Row():
        custom_message = gr.Textbox(
            label="Additional Notes for Discharge Letter",
            placeholder="Enter any additional instructions or notes for the discharge letter",
            lines=3
        )

    with gr.Row():
        generate_button = gr.Button("Generate Discharge Letter")

    pdf_output = gr.File(label="Download your Discharge Letter")

    # Update dropdown when JSON is uploaded
    def update_dropdown(file):
        if file is None:
            return [], {}
        patient_choices, patient_dict = load_patient_data(file)
        return gr.Dropdown(choices=patient_choices), patient_dict

    json_file.change(
        fn=update_dropdown,
        inputs=[json_file],
        outputs=[patient_dropdown, patient_data_dict]
    )

    # Generate PDF when button is clicked
    generate_button.click(
        fn=generate_pdf,
        inputs=[patient_dropdown, custom_message, patient_data_dict],
        outputs=pdf_output
    )

# Launch the app
if __name__ == "__main__":
    demo.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


: 